<a href="https://www.kaggle.com/code/jorgeluiscruzquispe/ex-leafmap-samgeo-code?scriptVersionId=166383039" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <center>Segment Anything Model for Geospatial Data</center>
---

reference: [https://github.com/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb](https://github.com/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)

In [ ]:
!pip install -q segment-geospatial

In [ ]:
%%capture
!conda install -y -c conda-forge gdal

In [ ]:
import os
import leafmap.foliumap as leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

In [ ]:
m = leafmap.Map(center=[29.676840, -95.369222], zoom=19)
m.add_basemap("SATELLITE")
m

In [ ]:
bbox = [-95.3704, 29.6762, -95.368, 29.6775]

image = "satellite.tif"

In [ ]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

In [ ]:
m = leafmap.Map(center=[29.676840, -95.369222], zoom=19)
# m.layer_opacity
# m.layers[-1].visible = False  # turn off the basemap
m.add_basemap("SATELLITE")
m.add_image(image)
# m.add_image
m

In [ ]:
sam = SamGeo(
    model_type = "vit_h",
    checkpoint = "sam_vit_h_4b8939.pth",
    sam_kwargs = None,
)

In [ ]:
mask = "segment.tif"
sam.generate(
    image, mask, batch = True, foreground = True, erosion_kernel = (3, 3), mask_multiplier = 255
)

In [ ]:
vector = "segment.gpkg"
sam.tiff_to_gpkg(mask, vector, simplify_tolerance = None)

In [ ]:
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

In [ ]:
m = leafmap.Map(center=[29.676840, -95.369222], zoom=19)
m.add_basemap("SATELLITE")
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}

m.add_vector(vector, layer_name = "Vector", style = style)
m